# Formula Exploration Notebook V2

This notebook holds the attempt to explore formulas in Sympy further, especially custom ones.

## Basic custom formula

In [2]:
from sympy.abc import x
from sympy.utilities.lambdify import lambdify, implemented_function
from sympy.parsing.sympy_parser import parse_expr

custom_func = implemented_function('custom_func', lambda x: x+1)
lambdified_custom_func = lambdify(x, custom_func(x))

functions_dict = {
    'custom_func': lambdified_custom_func
}

# Test parsing
result = parse_expr('custom_func(4)', local_dict=functions_dict)
print(f'Result of custom function: {result}')

# Yes, this is working!

Result of custom function: 5


In [3]:
# What about nested functions?

nested_result = parse_expr('custom_func(custom_func(4))', local_dict=functions_dict)
print(f'Result of nested function: {nested_result}')

# Yeah, this works!

Result of nested function: 6


In [7]:
# What about parsing a function with variable?


variable_result = parse_expr('custom_func(a_1)', local_dict=functions_dict)
print(f'Result of custom function with variable: {variable_result}')


# Can we pass values to the variable?

new_dict = {'a_1': 10}
new_dict.update(functions_dict)


replaced_variable_result = parse_expr('custom_func(a_1)', local_dict=new_dict)
print(f'Result of custom function with filled variable: {replaced_variable_result}')

# Awesome, this works!

Result of custom function with variable: a_1 + 1
Result of custom function with filled variable: 11
